In [ ]:
!pip install networkx
!pip install osmnx

In [42]:
import pickle
import networkx as nx
import osmnx as ox
import geopandas
import numpy as np

In [ ]:
with open('..\Data\Raw\Info_bici\strade_ciclabili.pickle', 'rb') as x:    
    G = (pickle.load(x))

In [23]:
gdf = ox.graph_to_gdfs(G, nodes=False)

In [ ]:
gdf.head
gdf["highway"].value_counts()

In [32]:
gdf_motorways = gdf[(gdf["highway"] == "motorway")]
len(gdf_motorways)

81

In [34]:
gdf_motorways = gdf[(gdf["highway"] == "motorway") | (gdf["highway"] == "motorway_link")]

print(len(gdf_motorways))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_motorways.to_file('..\\Data\\Raw\\Info_bici\\strade_motorway.geojson', driver='GeoJSON')

311


In [38]:
gdf_trunks = gdf[(gdf["highway"] == "trunk") | (gdf["highway"] == "trunk_link")]

print(len(gdf_trunks))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_trunks.to_file('..\\Data\\Raw\\Info_bici\\strade_trunk.geojson', driver='GeoJSON')

106


In [ ]:
gdf_primary = gdf[(gdf["highway"] == "primary") | (gdf["highway"] == "primary_link")]


print(len(gdf_primary))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_primary.to_file('..\\Data\\Raw\\Info_bici\\strade_primary.geojson', driver='GeoJSON')


In [44]:
# Sostituisci i None con np.nan nel GeoDataFrame primario
gdf_primary.loc[:, "maxspeed"] = gdf_primary["maxspeed"].replace({None: np.nan})

# Filtra solo le righe con maxspeed mancante
gdf_null_maxspeed = gdf_primary[gdf_primary["maxspeed"].isna()]

# Salva su file GeoJSON
gdf_null_maxspeed.to_file('..\\Data\\Raw\\Info_bici\\strade_primary_null.geojson', driver='GeoJSON')


In [ ]:
gdf_footway = gdf[(gdf["highway"] == "footway")]


print(len(gdf_footway))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_primary.to_file('..\\Data\\Raw\\Info_bici\\strade_footway.geojson', driver='GeoJSON')


In [ ]:
# Strade Secondary

gdf_secondary = gdf[(gdf["highway"] == "secondary")]
len(gdf_secondary)
print(gdf[gdf["highway"] == "secondary"].isna().sum())

gdf_secondary = gdf[(gdf["highway"] == "secondary") | (gdf["highway"] == "secondary_link")]

print(len(gdf_secondary))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_secondary.to_file("..\\Data\\Raw\\Info_bici\\strade_secondary.geojson", driver='GeoJSON')

In [ ]:
# Strade Tertiary

gdf_tertiary = gdf[(gdf["highway"] == "tertiary")]
len(gdf_secondary)
print(gdf[gdf["highway"] == "tertiary"].isna().sum())

gdf_tertiary = gdf[(gdf["highway"] == "tertiary") | (gdf["highway"] == "tertiary_link")]

# Salva in GeoJSON
gdf_tertiary.to_file("..\\Data\\Raw\\Info_bici\\strade_tertiary.geojson", driver='GeoJSON')

In [ ]:
# Residential

gdf_residential = gdf[(gdf["highway"] == "residential")]
print(gdf[gdf["highway"] == "residential"].isna().sum())

print(len(gdf_secondary))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_residential.to_file("..\\Data\\Raw\\Info_bici\\strade_residential.geojson", driver='GeoJSON')

In [ ]:
# Service

gdf_service = gdf[(gdf["highway"] == "service")]
print(gdf[gdf["highway"] == "service"].isna().sum())

print(len(gdf_service))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_service.to_file("..\\Data\\Raw\\Info_bici\\strade_service.geojson", driver='GeoJSON')

In [ ]:
# Creazione geodataframe manuale

import json
import geopandas as gpd
from shapely.geometry import shape
import numpy as np

# Carica il GeoJSON come dizionario
with open("Data/Staging/Info_bici/strade_ciclabili.geojson", "r", encoding="utf-8") as f:
    data = json.load(f)

# Costruisci il GeoDataFrame manualmente
features = data["features"]
geometries = [shape(feature["geometry"]) for feature in features]
properties = [feature["properties"] for feature in features]

gdf = gpd.GeoDataFrame(properties, geometry=geometries)

# Imposta il CRS (Coordinate Reference System)
gdf.set_crs("EPSG: 4326", inplace=True)

# 1. Assicuriamoci che valori None siano convertiti in np.nan
gdf['highway'] = gdf['highway'].replace({None: np.nan})

# 2. Se necessario, converti in stringa (utile se ci sono tipi misti)
gdf['highway'] = gdf['highway'].astype('object')

# 3. Crea la maschera con entrambe le condizioni:
mask = (
    gdf['highway'].isna() |
    (gdf['highway'] == 'unclassified')
)

# 4. Applica il filtro
gdf_filtered = gdf[mask]

gdf_filtered.duplicated()

gdf_filtered.to_file ("Data/Staging/Info_bici/highway_null_unclassified.geojson")